In [3]:
import openai
import os
import pinecone

from dotenv import load_dotenv

load_dotenv()
# environment variables

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG_KEY")

embed_model = "text-embedding-ada-002"

# Manually type some text
texts = [
    "Andy Appel knew a man named Joel Friedman that used to race aligators. He worked at bell labs and tested how oxygen is absorbed in the blood.",
    "Joel Friedman headed up the Albert Einstein shcool. Andy got to take care of the animals.",
]

# Generate embeddings using OpenAI API
res = openai.Embedding.create(input=texts, engine=embed_model)

# Extract embeddings
embeds = [record['embedding'] for record in res['data']]

# Initialize Pinecone
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment="us-central1-gcp")

index_name = 'memory'

# Create Pinecone index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )

# Connect to Pinecone index
index = pinecone.Index(index_name)

# Prepare metadata
metadata = [
    {"text": "Andy Appel knew a man named Joel Friedman that used to race alligators. He worked at bell labs and tested how oxygen is absorbed in the blood.",},
    {"text": "Joel Friedman headed up the Albert Einstein shcool. Andy got to take care of the animals."},
]

# Combine item IDs, embeddings, and metadata
item_ids = ["item1", "item2"]
embeddings_with_metadata = [
    {"id": item_id, "values": embed, "metadata": meta}
    for item_id, embed, meta in zip(item_ids, embeds, metadata)
]

# Upsert embeddings with metadata into Pinecone
index.upsert(embeddings_with_metadata)

{'upserted_count': 2}